<a id='data-structure'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Data Model </span> </p>

This notebook has the main structures used to hold data - This is usually called the **data model**.

# References
This section initializes libraries and code from other notebooks, which will be used here, as well as information needed for querying data from the database.

## Libraries

In [0]:
#r "nuget:Systemorph.Activities,1.5.7"
#r "nuget:Systemorph.Arithmetics,1.5.7"
#r "nuget:Systemorph.Workspace,1.5.4"
#r "nuget:Systemorph.InteractiveObjects,1.5.7"
#r "nuget:Systemorph.SharePoint,1.5.7"
#r "nuget:Systemorph.OneDrive,1.5.7"
#r "nuget:Systemorph.Scopes,1.5.7"
#r "nuget:Systemorph.Import,1.5.2"
#r "nuget:Systemorph.Test,1.5.7"
#r "nuget:Systemorph.Export,1.5.2"
#r "nuget:Systemorph.DataSetReader,1.5.2"
#r "nuget:Systemorph.DataSource,1.5.4"
#r "nuget:Systemorph.DataSource.Conversions,1.5.4"
#r "nuget:Systemorph.Reporting,1.5.7"

## Usings

In [0]:
using System.ComponentModel.DataAnnotations;
using System.ComponentModel.DataAnnotations.Schema;
using Systemorph.Vertex.Grid.Model;
using Systemorph.Vertex.Workspace;
using Systemorph.Vertex.Activities;
using Systemorph.Vertex.Import;
using static Systemorph.Vertex.Arithmetics.ArithmeticOperations;

## Notebooks

In [0]:
#!import "../Constants/Enums"
#!import "../Constants/Consts"
#!import "../Constants/Validations"

# Data Infrastructure

<a id='base-interfaces'></a>
## Base Interfaces
The following interfaces are used to define the granularity at which data is stored.

For instance, the IKeyed interface corresponds to the lowest granularity which is simply a unique identifier for a certain generic piece of data:

In [0]:
public interface IKeyed
{   
    public Guid Id { get; init; }
}

The interface IPartition identifies the partitions in the system.

In [0]:
public interface IPartition : IKeyed {}

The interface IPartitioned simply assigns a unique identifier for elements of a data partition - The exact definition of such elements can be found in the [partition](#partition) section below.

In [0]:
public interface IPartitioned
{
    public Guid Partition { get; init; }
}

The interface IHierarchy is used for modelling dimensions which have an hierarchical structure, such as [Amount Type](#amount-type).

In [0]:
public interface IHierarchy
{
    public string Name { get; init; }
 
    public string Parent { get; init; }
    
    public string Child { get; init; }
}

The interface IWithYearAndMonth allows to speficy to which year and month a certain piece of data applies to:

In [0]:
public interface IWithYearAndMonth
{
    public int Year { get; init; }
    
    public int Month { get; init; }
}

The interface IWithYearMonthAndScenario allows to identify the year, month and scenario associated to the data:


In [0]:
public interface IWithYearMonthAndScenario : IWithYearAndMonth
{
    public string Scenario { get; init; }
}

<a id='abstract-classes'></a>
## Abstract Classes
These classes are not reflected in the database, i.e. are not persisted.

In [0]:
public abstract record KeyedRecord : IKeyed {
    [Key]
    [NotVisible]     
    public Guid Id { get; init; }
}

In [0]:
public abstract record KeyedDimension : INamed {
    [Key]
    [IdentityProperty]
    [StringLength(16)]
    [Display(Order = 1)]
    public string SystemName { get; init; }
    
    [Display(Order = 2)]
    [NotVisible]
    public string DisplayName { get; init; }
}

In [0]:
public abstract record KeyedOrderedDimension : KeyedDimension, IOrdered {
    [NotVisible]
    public int Order { get; init; }
}

In [0]:
public abstract record KeyedOrderedDimensionWithExternalId : KeyedOrderedDimension {
    [Display(Order = 100)]
    [Conversion(typeof(JsonConverter<string[]>))]
    public string[] ExternalId { get; init; }
}

<a id='dimensions'></a>
# Dimensions

Dimensions are data which tipically can be given a short name, here called 'System Name'. They also change slowly with time. A typical example is currencies, and examples of System Names are EUR, CHF, etc.

The advantages of using Dimensions are:
- The data is easier to maintain (the data is said to be 'normalized');
- The business logic is easier to read;
- Easier scructuring of the data, e.g. in a data-cube.

<a id='hierarchical-dimension-with-level'></a>
## Hierarchical Dimension With Level
Hierarchical dimensions are useful to define for example the legal structure of a company (reporting nodes and their parents).

Below we introduce the concept of a hierarchical dimension with level, which is a simple way to keep track of the "level" in a hierarchical dimension.

In [0]:
//TODO: Should this be abstract or have a reference to the type?
public record HierarchicalDimensionWithLevel(string SystemName, string DisplayName, string Parent, int Level) : IHierarchicalDimension;

<a id='amount-type'></a>
## Amount Type

Amount Type is a dimension used to describe the nature of numbers. For example: Numbers associated to the payment of premiums have Amount Type 'Premiums'; Numbers associated to the payment of claims have Amount Type 'Claims'; Numbers associated to expenses have Amount Type 'Expenses'. 

It is defined as a hierarchical dimension so that higher-granularity types can be introduced. For example: 'Claim Expenses' have parent 'Claims'.

Amount Types are a central piece in the IFRS 17 application and are used for instance to name the different types of cashflows. For example: There might be cashflows for 'Premiums'; Cashflows for 'Claims', etc. In case the mapping between cash flow and amount type is non-trivial, then specific mappings are introduced through the use of <code>ExternalId</code>.

In [0]:
public record AmountType : KeyedOrderedDimensionWithExternalId, IHierarchicalDimension
{
    [Display(Order = 10)]
    [Dimension(typeof(AmountType))]
    public string Parent { get; init; }
    
    [Display(Order = 20)]
    [Dimension(typeof(PeriodType))]
    public PeriodType PeriodType { get; init; }
}

We distinguish between amount types used for cashflows and actuals (**Present Value Amount Types**), and amount types used for deferrable values (**Deferrable Amount Types**). In particular, deferrable amount types are for costs and expenses that can be deferred to the future. These amounts are treated separately from the Present Value Amount Types because they are not considered for the calculation of Contractual Service Margin and Loss Component.

In [0]:
public record DeferrableAmountType : AmountType {}

<a id='risk-driver'></a>
## Risk Driver

Risk drivers are used to distinguish between the different Risk Adjustment numbers. For instance: Mortality, Disability, Longevity, etc.

When only total risk adjustment values are used, then there is no need to use risk drivers.

In [0]:
public record RiskDriver : KeyedOrderedDimension, IHierarchicalDimension
{
    [Display(Order = 10)]
    [Dimension(typeof(RiskDriver))]
    public string Parent { get; init; }
}

<a id='estimate-type'></a>
## Estimate Type

Estimate Type is a <href >dimension used to distinguish the context to which numbers belong to. For instance: Numbers for present values have Estimate Type 'Present Value'; Numbers for Risk Adjustments have Estimate Type 'Risk Adjustment'.

In [0]:
public record EstimateType : KeyedOrderedDimensionWithExternalId
{
    [Display(Order = 110)]
    public InputSource InputSource { get; init; }
    
    [Display(Order = 120)]
    public StructureType StructureType { get; init; }

    [Dimension(typeof(PeriodType))]
    public PeriodType PeriodType { get; init; }
}

<a id='novelty'></a>
## Novelty

Novelty is a [dimension](#dimensions) used to identity the novelty of the business. Typical examples of Novelty are New Busines, In-Force, and Combined.

In [0]:
public record Novelty : KeyedOrderedDimension {}

<a id='variable-type'></a>
## Variable Type

In [0]:
public record VariableType : KeyedOrderedDimension, IHierarchicalDimension
{    
    [Display(Order = 10)]
    public string Parent { get; init; }
}

<a id='aoc-type'></a>
### AoC Variable Type

The AocType describes the type of the Analysis of Change step - For example: Amortization, Cashflow, Combined Liabilities etc.

The **Combined Liability** AocType aims in capturing the difference between the sum of all relevant previous Aoc steps and the end of period Aoc step. Unexplained changes (not captured by any other defined AoC step), are then visible in this Aoc step. Note that the nominal cash flows input in the cashflow input file corresponds to cash flow projection for the end of period step.
In addition, this step distinguishes itself from the others for:
- It is a mandatory AocType as it is used to trigger the Run off of the cashflow. This is achieved by providing a cash flow of 0s. 
- It is the first AocType for the Combined Novelty and therefore it carries the contribution of combining In force and New business.
- Its present value is computed as telescoping difference with the last AoC Step for the In-Force novelty and the New Business novelty.
- In the technical margin calculation it is used to merge the profitability of the In-Force and New business components, which may result in a forced switch.

In [0]:
public record AocType : VariableType
{    
    [Display(Order = 30)]
    [Dimension(typeof(AocType))]
    public string Parent { get; init; }
}

In [0]:
public record AocStep(string AocType, string Novelty){}

<a id='pnl-variable-type'></a>
### PnL Variable Type

The PnlVariableType is used to describe the various profit and loss sections and the corresponding hierarchy.

In [0]:
public record PnlVariableType : VariableType {}

<a id='bs-variable-type'></a>
## Balance Sheet Variable Type

The BsVariableType is used to descibe the Anlysis of change of the Balance Sheet reports.

In [0]:
public record BsVariableType : VariableType {}

## Accounting Variable Type
The AccountingVariableType is used to describe the accounting events that are posted in the accounting output file.

In [0]:
public record AccountingVariableType : VariableType {}

<a id='scenario'></a>
## Scenario
The <code>Scenario</code> record holds the various scenarios for which calculations should also be performed.

In [0]:
public record Scenario : KeyedDimension {}

 The 'neutral' Scenario (i.e. the default one, with no-stress situations applied) is designated as 'Best Estimate'.

<a id='line-of-business'></a>
## Line Of Business

Line Of Business is a dimension used to identity insurance business according to usual business classifications (usually a market or company-specific business classification representing a convolution of risk covered, 1st party / 3rd party cover type, product type, etc...)
Typical examples of Line of Business are Motor, Property, Endowment, etc... This is a hierarchical dimension.

In [0]:
public record LineOfBusiness : KeyedOrderedDimension, IHierarchicalDimension
{
    [Display(Order = 10)]
    [Dimension(typeof(LineOfBusiness))]
    public string Parent { get; init; }
}

<a id='currency'></a>
## Currency

Currency is a dimension used to identity the currency.
Typical examples of Currency are EUR, CHF, USD, etc...

In [0]:
public record Currency : KeyedDimension {}

<a id='economic-basis'></a>
## Economic Basis

The Economic Basis defines the discount type which is used in calculations. Examples are: Nominal, Current and Locked.

In [0]:
public record EconomicBasis : KeyedDimension {}

<a id='valuation-approach'></a>
## Valuation Approach
This dimension holds the various methodologies used in the calculations: BBA, VFA and PAA.

In [0]:
public record ValuationApproach : KeyedDimension {}

<a id='liability-type'></a>
## Liability Type
The Liability Type distinguishes betweeen e.g. liabilities for incurred claims and liabilities for remaining coverage.

In [0]:
public record LiabilityType  : KeyedDimension, IHierarchicalDimension
{
    [Display(Order = 10)]
    [Dimension(typeof(LiabilityType))]
    public string Parent { get; init; }
}

<a id='oci-type'></a>
## OCI Type
Other Comprehensive Income (OCI) Type describes how the financial contributions to the IFRS17 Financial Performance is apportioned between the P&L and the OCI sections.

In [0]:
public record OciType : KeyedDimension {}

<a id='profitability'></a>
## Profitability

Profitability is a dimension used in IFRS 17 to describe the profitability of a Group of Contracts (GIC/GRIC).
Typical examples of Profitability are Profitable, Non-Profitable, etc..

In [0]:
public record Profitability : KeyedDimension {}

<a id='partner'></a>
## Partner
Partner is the counterparty of a reinsurance contract.

In [0]:
public record Partner : KeyedDimension {}

<a id='credit-risk-rating'></a>
## Credit Risk Rating
This holds the credit risk rating of each Partner.

In [0]:
public record CreditRiskRating : KeyedDimension {}

<a id='reporting-node'></a>
## Reporting Node
A Reporting Node is an element in the reporting structure of the company. It has a hierarchical structure and the end nodes is the level at which data is imported.

In [0]:
public record ReportingNode : KeyedDimension, IHierarchicalDimension 
{
    [Display(Order = 20)]
    [Dimension(typeof(ReportingNode))]
    public string Parent { get; init; }
    
    [Required]
    [Display(Order = 40)]
    [Dimension(typeof(Currency))]
    public virtual string Currency { get; init; }
}

<a id='projection-configuration'></a>
## Projection Configuration

Projections are configured by defining the following 2 parameters:

- <b>Shift</b> defines how many months are in the cashflow file before the first value of the period in interest.<br> 
- <b>TimeStep</b> defines how many months are in the period of interest. <br>

Both <b>Shift</b> and <b>TimeStep</b> are measured in terms of number of months.
For instance, for the current year and year-to-date view we have $\rm{Shift}=0$ and $TimeStep=3$ for the first quarter, $TimeStep=6$ for the 2nd quarter and so on.

In [0]:
public record ProjectionConfiguration : KeyedDimension
{
    [IdentityProperty]
    public int Shift { get; init; }
    [IdentityProperty]
    public int TimeStep { get; init; }
}

<a id='parameters'></a>
# Parameters

<a id='aoc-configuration'></a>
## Aoc Step Configuration

The record below holds parameters over time for each combination of AocType and Novelty. In particular:
- <code>DataType</code> identifies between Calculated aoc steps and Optional inputs or Mandatory inputs.
- <code>InputSource</code> identifies whether an aoc step is relevant for Actuals, Cashflow or both calculations.
- <code>FxPeriod</code> identifies which fx rate should be used.
- <code>YcPeriod</code> identifies which Yield Curve should be used (from current or previous period).
- <code>CdrPeriod</code> identifies which Credit Default Rate should be used (from current or previous period).
- <code>ValuationPeriod</code> identifies which value from the Cashflow corresponds to the Present Value.
- <code>RcPeriod</code> identifies which Reinsurance Coverage should be used (from current or previous period).

<br>The values for each parameter can be found in [Enums Notebook](./Constants/Enums).

In [0]:
public record AocConfiguration : KeyedRecord, IWithYearAndMonth, IOrdered
{
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 10)]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 20)]
    public int Month { get; init; }
    
    [IdentityProperty]
    [Display(Order = 30)]
    [Dimension(typeof(AocType))]
    public string AocType { get; init; }
    
    [IdentityProperty]
    [Display(Order = 40)]
    [Dimension(typeof(Novelty))]
    public string Novelty { get; init; }
    
    [Display(Order = 50)]
    [Dimension(typeof(DataType))]
    public DataType DataType { get; init; }
    
    [Display(Order = 55)]
    [Dimension(typeof(InputSource))]
    public InputSource InputSource { get; init; }
    
    [Display(Order = 60)]
    [Dimension(typeof(FxPeriod))]
    public FxPeriod FxPeriod { get; init; }
    
    [Display(Order = 70)]
    [Dimension(typeof(PeriodType), nameof(YcPeriod))]
    public PeriodType YcPeriod { get; init; }
    
    [Display(Order = 80)]
    [Dimension(typeof(PeriodType), nameof(CdrPeriod))]
    public PeriodType CdrPeriod { get; init; }
    
    [Display(Order = 90)]
    [Dimension(typeof(ValuationPeriod))]
    public ValuationPeriod ValuationPeriod { get; init; }
    
    [Display(Order = 100)]
    [Dimension(typeof(PeriodType), nameof(RcPeriod))]
    public PeriodType RcPeriod { get; init; }
    
    [NotVisible]
    public int Order { get; init; }
}

<a id='exchange-rate'></a>
## Exchange Rate
The record below holds the Exchange Rates over time and for each exchange type ([FxType](./Constants/Enums)). Exchange Rate must be updated every reporting period as part of the close preparation. In general, the last month of the reporting period should be use to import the new data for the period. 

In case the Exchange Rate for the current period is missing, an error is returned. 

In [0]:
public record ExchangeRate : KeyedRecord, IWithYearMonthAndScenario
{    
    [Required]
    [IdentityProperty]
    [Display(Order = 10)]
    [Dimension(typeof(Currency))]
    public string Currency { get; init; }

    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 20)]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 30)]
    public int Month { get; init; }

    [IdentityProperty]
    [Required]
    [Display(Order = 40)]
    public FxType FxType { get; init; }

    [Display(Order = 50)]
    public double FxToGroupCurrency { get; init; }

    [IdentityProperty]
    [Display(Order = 60)]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }
}

<a id='credit-default-rate'></a>
## Credit Default Rate
The record below holds the Credit Default Rates over time for each [Credit Risk Rating](#credit-risk-rating). Credit Default Rate can be updated every month as part of the close preparation. In general, the last month of the reporting period should be use to import the new data for the period. 

The Credit Default Rate values must be entered with monthly granularity, i.e. every value corresponds to the default rate for a certain month. The first month entered in column Values0 always corresponds to January of the year entered in column <code>Year</code>. The last value is considered valid also for the following years (for which no value has been provided). 

The latest Credit Default Rate available in the system with Year and Month earlier or equal to Year and Month of the closing period will be used as the current value during calculation.

In [0]:
public record CreditDefaultRate : KeyedRecord, IWithYearMonthAndScenario
{    
    [Required]
    [IdentityProperty]
    [Display(Order = 10)]
    [Dimension(typeof(CreditRiskRating))]
    public string CreditRiskRating { get; init; }

    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 20)]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 30)]
    public int Month { get; init; }
    
    [Conversion(typeof(PrimitiveArrayConverter))]
    [Display(Order = 40)]
    public double[] Values { get; init; }

    [IdentityProperty]
    [Display(Order = 50)]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }
}

<a id='yield-curve'></a>
## Yield Curve
The record below holds the Yield Curves across time for each [Currency](#currency) and [Scenario](#scenario).
Yield curves can be updated every month as part of the close preparation.  In general, the last month of the reporting period should be use to import the new data for the period. 

The Yield Curve values must be entered with **yearly** granularity, i.e. every value corresponds to the rate of the whole year starting from the year entered in column <code>Year</code>. The last value is considered valid also for the following years (for which no value has been provided). 

When the current Yield Curve must be used in the calculation, the latest Yield Curve available in the system with Year and Month earlier or equal to Year and Month of the closing period will be used.

In [0]:
public record YieldCurve : KeyedRecord, IWithYearMonthAndScenario
{    
    [Required]
    [IdentityProperty]
    [Display(Order = 10)]
    [Dimension(typeof(Currency))]
    public string Currency { get; init; }

    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 20)]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 30)]
    public int Month { get; init; }

    [IdentityProperty]
    [Display(Order = 35)]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }

    [Conversion(typeof(PrimitiveArrayConverter))]
    [Display(Order = 40)]
    public double[] Values { get; init; }
}

<a id='partner-rating'></a>
## Partner Rating
The record below holds the [Credit Risk Rating](#credit-risk-partner) for each [Partner](#partner) across time. Partner Rating can be updated every month as part of the close preparation. In general, the last month of the reporting period should be use to import the new data for the period. 

In [0]:
public record PartnerRating : KeyedRecord, IWithYearMonthAndScenario
{    
    [Required]
    [IdentityProperty]
    [Display(Order = 10)]
    [Dimension(typeof(Partner))]
    public string Partner { get; init; }

    [Required]
    [Display(Order = 20)]
    [Dimension(typeof(CreditRiskRating))]
    public string CreditRiskRating { get; init; }

    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 30)]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 40)]
    public int Month { get; init; }

    [IdentityProperty]
    [Display(Order = 50)]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }
}

<a id='partitions'></a>
# Partitions
The notion of Partition is used to define small but meaningful sets of data, called partitions. Partitions are then used to optimize the extraction of data from the database, i.e. only the data for a certain partition is retrieved.

In [0]:
public abstract record IfrsPartition : IPartition {
    [Key]
    [PartitionId]
    public Guid Id { get; init; }

    [Required]
    [Display(Order = 10)]
    [Dimension(typeof(ReportingNode))]
    [IdentityProperty]
    public string ReportingNode { get; init; }

    [Display(Order = 50)]
    [Dimension(typeof(Scenario))]
    [IdentityProperty]
    public string Scenario { get; init; }
}

For instance, the data pertaining to a certain [Reporting Node](#reporting-node) for a certain [Scenario](#scenario) is called an <code>IfrsPartition</code> and assigned a unique identifier:

In [0]:
public record PartitionByReportingNode : IfrsPartition {}

The partition </code>PartitionByReportingNodeAndPeriod</code> is a further partition of the <code>IfrsPartition</code> sets - In particular, it defines sets for the  data pertaining to a certain [Reporting Node](#reporting-node), [Scenario](#scenario), year and month. The value of the <code>Month</code> is the last month of the reporting period to which the data belongs to. 

In [0]:
public record PartitionByReportingNodeAndPeriod : IfrsPartition {
    [Display(Order = 30)]
    [Dimension(typeof(int), nameof(Year))]
    [IdentityProperty]
    public int Year { get; init; }

    [Display(Order = 40)]
    [Dimension(typeof(int), nameof(Month))]
    [IdentityProperty]
    public int Month { get; init; }
}

<a id='policy-related-data-structures'></a>
# Policy-related Data Structures

The data structures below are the pieces of the data model where policy-related data is held.

<a id='data-node'></a>
## Data Node

A data node is basically a set of labels which identify data according to its:
- Reporting Node
- Scenario
- Contractual Currency
- FunctionalCurrency
- LineOfBusiness
- ValuationApproach
- OCI Type

In [0]:
public record DataNode : KeyedDimension, IPartitioned {
    [NotVisible]
    [PartitionKey(typeof(PartitionByReportingNode))]
    public Guid Partition { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(Currency))]
    //[Immutable]
    public string ContractualCurrency { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(Currency))]
    //[Immutable]
    public string FunctionalCurrency { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(LineOfBusiness))]
    //[Immutable]
    public string LineOfBusiness { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(ValuationApproach))]
    [Required]
    //[Immutable]
    public string ValuationApproach { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(OciType))]
    //[Immutable]
    public string OciType { get; init; }
}

<a id='portfolios'></a>
## Portfolios

A Portfolio is a set of Group of Contracts. The relationship is set on the Group of Contracts (see below).

Portfolios are of type DataNode and there are two types: Insurance Portfolio and Reinsurance Portfolios.

In [0]:
public record Portfolio : DataNode {}

public record InsurancePortfolio : Portfolio {}
public record ReinsurancePortfolio : Portfolio {}

<a id='group-of-contracts'></a>
## Group of Contracts

A Group of Contract is a set of contracts with the same Annual Cohort, Liability Type, Profitability and Partner (in case the contracts are done with a re-insurer).

Various Group of Contracts are put together into portfolios.

In [0]:
public record GroupOfContract : DataNode {
    [NotVisible]    
    [Dimension(typeof(int), nameof(AnnualCohort))]
    //[Immutable]
    public int AnnualCohort { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(LiabilityType))]
    //[Immutable]
    public string LiabilityType { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(Profitability))]
    //[Immutable]
    public string Profitability { get; init; }
 
    [Required]
    [NotVisible]    
    [Dimension(typeof(Portfolio))]
    //[Immutable]
    public string Portfolio { get; init; }
    
    public virtual string Partner { get; init; }
}

A group of insurance contracts is called a **GIC** and has no information on Partner.

Only Groups of Reinsurance Contracts, defined below and usually called **GRIC**s, have Partner information.
Again, there are Insurance GRICs and Reinsurance GRICs:

In [0]:
public record GroupOfInsuranceContract : GroupOfContract {
    [Required]
    [NotVisible]    
    [Display(Name = "InsurancePortfolio")]
    [Dimension(typeof(InsurancePortfolio))]
    //[Immutable]
    public string Portfolio { get => base.Portfolio; init => base.Portfolio = value; }
    
    // TODO: for the case of internal reinsurance the Partner would be the reporting node, hence not null.
    // If this is true we need the [Required] attribute here, add some validation at dataNode import 
    // and to add logic in the GetNonPerformanceRiskRate method in ImportStorage.
    [NotVisible]    
    [NotMapped]
    //[Immutable]
    public override string Partner => null;
}

public record GroupOfReinsuranceContract : GroupOfContract {
    [Required]
    [NotVisible]    
    [Display(Name = "ReinsurancePortfolio")]
    [Dimension(typeof(ReinsurancePortfolio))]
    //[Immutable]
    public string Portfolio { get => base.Portfolio; init => base.Portfolio = value; }
}

<a id='data-node-state'></a>
## Data Node State

The Data State is used to track whether a GOC is Active or Inactive across time (year and month). 
It is only possible to update the state of a Group of Contract from Active to Inactive.

In [0]:
public record DataNodeState : KeyedRecord, IPartitioned, IWithYearMonthAndScenario
{
    [NotVisible]
    [PartitionKey(typeof(PartitionByReportingNode))]
    public Guid Partition { get; init; }
    
    [Required]
    [IdentityProperty]
    [Display(Order = 10)]
    [Dimension(typeof(GroupOfContract))]
    public string DataNode { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 20)]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 30)]
    [DefaultValue(DefaultDataNodeActivationMonth)]
    public int Month { get; init; } = DefaultDataNodeActivationMonth;
    
    [Required]
    [Display(Order = 40)]
    [DefaultValue(State.Active)]
    public State State { get; init; } = State.Active;

    [IdentityProperty]
    [Display(Order = 50)]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }
}

<a id='data-node-parameters'></a>
## Data Node Parameters

Data Node Parameters are used to keep track of other parameters pertaining to each Data Node, and their movements in time (year and month).
<br> <code>PremiumAllocation</code> : defines the weight of Premium to be included in the Experience Adjustement AocType of the Technical Margin and is valid only for Group of Insurance Contract with LiabilityType : Liability for Remaining Coverage.
<br> <code>ReinsuranceCoverage</code> : defines the weight of the underlying gross business to be considered in the computation of the allocation of the Technical Margin in a Reinsurance case.

The latest Data Node Parameters available in the system with Year and Month earlier or equal to Year and Month of the closing period will be used as the current value during calculation.

In [0]:
public record DataNodeParameter : KeyedRecord, IPartitioned, IWithYearMonthAndScenario
{
    [NotVisible]
    [PartitionKey(typeof(PartitionByReportingNode))]
    public Guid Partition { get; init; }
        
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Year))]
    [Range(1900, 2100, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 20)]
    public int Year { get; init; }
    
    [Required]
    [IdentityProperty]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Month))]
    [Range(1, 12, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    [Display(Order = 30)]
    [DefaultValue(DefaultDataNodeActivationMonth)]
    public int Month { get; init; } = DefaultDataNodeActivationMonth;
        
    [Required]
    [IdentityProperty]
    [Display(Order = 10)]
    [Dimension(typeof(GroupOfContract))]
    public string DataNode { get; init; }

    [IdentityProperty]
    [Display(Order = 40)]
    [Dimension(typeof(Scenario))]
    public string Scenario { get; init; }
}

In [0]:
public record SingleDataNodeParameter : DataNodeParameter {
    [DefaultValue(DefaultPremiumExperienceAdjustmentFactor)]
    [Range(0, 1, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public double PremiumAllocation { get; init; } = DefaultPremiumExperienceAdjustmentFactor;
}

public record InterDataNodeParameter : DataNodeParameter {
    [Required]
    [IdentityProperty]
    [Display(Order = 20)]
    [Dimension(typeof(GroupOfContract))]
    public string LinkedDataNode { get; init; }
    
    [Range(0, 1, ErrorMessage = "Value for {0} must be between {1} and {2}.")]
    public double ReinsuranceCoverage { get; init; }
}

## DataNodeData PocoType
The <code>DataNodeData</code> type is not persisted (i.e. not stored in the database) and is used in order to hold Data Node data during data manipulations.

In [0]:
public record DataNodeData {
    public string DataNode { get; init; }
    
    //Portfolio
    public string ContractualCurrency { get; init; }
    public string FunctionalCurrency { get; init; }
    public string LineOfBusiness { get; init; }
    public string ValuationApproach { get; init; }
    public string OciType { get; init; }
    
    //GroupOfContract
    public string Portfolio { get; init; }
    public int AnnualCohort { get; init; }
    public string LiabilityType { get; init; }
    public string Profitability { get; init; }
    public string Partner { get; init; }
    
    //DataNodeState
    public int Year { get; init; }
    public int Month { get; init; }
    public State State { get; init; }
    public State PreviousState { get; init; }
    
    public bool IsReinsurance { get; init; }
    public string Scenario { get; init; }
    public DataNodeData(){}
}

<a id='raw-variables'></a>
## Raw Variables (Cashflows)

Raw Variables are used to store **cashflows** on the database during import, i.e. *after* mappings are applied and *before* calculations are applied.

They are stored on the database for both audit and re-calculation purposes.

In [0]:
public abstract record BaseVariableIdentity {
    [NotVisible]
    [Dimension(typeof(GroupOfContract))]
    [IdentityProperty]
    public string DataNode { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(AocType))]
    [IdentityProperty]
    public string AocType { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(Novelty))]
    [IdentityProperty]
    public string Novelty { get; init; }
}

In [0]:
public abstract record BaseDataRecord : BaseVariableIdentity, IKeyed, IPartitioned {
    [Key]
    [NotVisible]     
    public Guid Id { get; init; }
    
    [NotVisible]
    [PartitionKey(typeof(PartitionByReportingNodeAndPeriod))]
    public Guid Partition { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(AmountType))]
    [IdentityProperty]
    public string AmountType { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(int),nameof(AccidentYear))]
    [IdentityProperty]
    public int? AccidentYear { get; init; }
}

Basically, <code>RawVariable</code> is defined for a certain Reporting Node, Scenario, Year, Month, Amount Type and Calculation Type:

In [0]:
public record RawVariable : BaseDataRecord
{
    [Conversion(typeof(PrimitiveArrayConverter))]
    public double[] Values { get; init; }
    
    [NotVisible]
    [Dimension(typeof(EstimateType))]
    [IdentityProperty]
    public string EstimateType { get; init; }
}

<a id='ifrs-variable'></a>
## Ifrs Variable

Ifrs Variables are used to store:

- **Actual Values**
- **Present Values**

Present Values are calculated from the [raw variables](#raw-variables) during the import of cashflows.

Ifrs Variables are stored on the database, i.e. they are 'persisted', for performance purposes.

In [0]:
public record IfrsVariable : BaseDataRecord
{
    public double Value { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(EstimateType))]
    [IdentityProperty]
    public string EstimateType { get; init; }
    
    [NotVisible]    
    [Dimension(typeof(EconomicBasis))]
    [IdentityProperty]
    public string EconomicBasis { get; init; }
    
    public IfrsVariable (){}
}

<a id='import-identity'></a>
## Import Identity
The Import Identity contains the data elements used to determine the data sets which are used when performing data manipulations, e.g. during a [data import](./Import/ImportScopeCalculation) - In particular, an Import Identity is defined by: DataNode, AocType, Novelty, and whether it's reinsurance data or not:

In [0]:
public record ImportIdentity : BaseVariableIdentity {
      
    [NotVisible]
    public bool IsReinsurance { get; init; }
    
    [NotVisible]
    public string ValuationApproach { get; init; }
    
    [NotVisible]
    public int ProjectionPeriod { get; init; }
    
    public (string, string) AocStep => (AocType, Novelty);
    
    public ImportScope ImportScope { get; init; }
    
    public ImportIdentity(RawVariable rv){
        DataNode = rv.DataNode;
        AocType = rv.AocType;
        Novelty = rv.Novelty;
    }
    
    public ImportIdentity(IfrsVariable iv){
        DataNode = iv.DataNode;
        AocType = iv.AocType;
        Novelty = iv.Novelty;
    }

    public ImportIdentity(){}
}

<a id='report-variable'></a>
# Report Variable

Report Variables are used to hold data after data manipulations and before being outputed in the form of a report. They are not persisted, i.e. they are not saved on the database.

In [0]:
public record ReportVariable {

    [NotVisible]
    [Dimension(typeof(ReportingNode))]
    [IdentityProperty]
    public string ReportingNode { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Scenario))]
    [IdentityProperty]
    public string Scenario { get; init; }

    [NotVisible]
    [Dimension(typeof(Currency))]
    [IdentityProperty]
    [AggregateBy]
    public string Currency { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Currency), nameof(FunctionalCurrency))]
    [IdentityProperty]
    public string FunctionalCurrency { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Currency), nameof(ContractualCurrency))]
    [IdentityProperty]
    public string ContractualCurrency { get; init; }
    
    [NotVisible]
    [Dimension(typeof(GroupOfContract))]
    [IdentityProperty]
    public string GroupOfContract { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Portfolio))]
    [IdentityProperty]
    public string Portfolio { get; init; }
    
    [NotVisible]
    [Dimension(typeof(LineOfBusiness))]
    [IdentityProperty]
    public string LineOfBusiness { get; init; }
    
    [NotVisible]
    [Dimension(typeof(LiabilityType))]
    [IdentityProperty]
    public string LiabilityType { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Profitability), nameof(InitialProfitability))]
    [IdentityProperty]
    public string InitialProfitability { get; init; }
    
    [NotVisible]
    [Dimension(typeof(ValuationApproach))]
    [IdentityProperty]
    public string ValuationApproach { get; init; }
    
    [NotVisible]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(AnnualCohort))]
    [IdentityProperty]
    public int AnnualCohort { get; init; }
    
    [NotVisible]
    [Dimension(typeof(OciType))]
    [IdentityProperty]
    public string OciType { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Partner))]
    [IdentityProperty]
    public string Partner { get; init; }
        
    [NotVisible]
    [IdentityProperty]
    public bool IsReinsurance { get; init; }
    
    [NotVisible]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(AccidentYear))]
    [IdentityProperty]
    public int AccidentYear { get; init; }
    
    [NotVisible]
    [NoArithmetics(ArithmeticOperation.Scale)]
    [Dimension(typeof(int), nameof(Projection))]
    [IdentityProperty]
    //[AggregateBy]
    public int Projection { get; init;}
    
    [NotVisible]
    [Dimension(typeof(VariableType))]
    [IdentityProperty]
    public string VariableType { get; init; }
    
    [NotVisible]
    [Dimension(typeof(Novelty))]
    [IdentityProperty]
    public string Novelty { get; init; }
    
    [NotVisible]
    [Dimension(typeof(AmountType))]
    [IdentityProperty]
    public string AmountType { get; init; }
    
    [NotVisible]
    [Dimension(typeof(EstimateType))]
    [IdentityProperty]
    public string EstimateType { get; init; }
    
    [NotVisible]
    [Dimension(typeof(EconomicBasis))]
    [IdentityProperty]
    public string EconomicBasis { get; init; }
    
    public double Value { get; init; }
    
    public ReportVariable(){}
    public ReportVariable(ReportVariable rv){
        ReportingNode = rv.ReportingNode;
        Scenario = rv.Scenario;
        Currency = rv.Currency;
        FunctionalCurrency = rv.FunctionalCurrency;
        ContractualCurrency = rv.ContractualCurrency;
        GroupOfContract = rv.GroupOfContract;
        Portfolio = rv.Portfolio;
        LineOfBusiness = rv.LineOfBusiness;
        LiabilityType = rv.LiabilityType;
        InitialProfitability = rv.InitialProfitability;
        ValuationApproach = rv.ValuationApproach;
        AnnualCohort = rv.AnnualCohort;
        OciType = rv.OciType;
        Partner = rv.Partner;
        IsReinsurance = rv.IsReinsurance;
        AccidentYear = rv.AccidentYear;
        Projection = rv.Projection;
        VariableType = rv.VariableType;
        Novelty = rv.Novelty;
        AmountType = rv.AmountType;
        EstimateType = rv.EstimateType;
        EconomicBasis = rv.EconomicBasis;
        Value = rv.Value;
    }
    public ReportVariable(DataNodeData dn, IfrsVariable iv){
        FunctionalCurrency  = dn.FunctionalCurrency;
        ContractualCurrency  = dn.ContractualCurrency;
        GroupOfContract = dn.DataNode;
        Portfolio = dn.Portfolio;
        LineOfBusiness  = dn.LineOfBusiness;
        LiabilityType  = dn.LiabilityType;
        InitialProfitability  = dn.Profitability;
        ValuationApproach  = dn.ValuationApproach;
        AnnualCohort  = dn.AnnualCohort;
        OciType  = dn.OciType;
        Partner  = dn.Partner;
        IsReinsurance  = dn.IsReinsurance;
        AccidentYear  = iv.AccidentYear ?? default;
        VariableType  = iv.AocType;
        Novelty  = iv.Novelty;
        AmountType  = iv.AmountType;
        EstimateType  = iv.EstimateType;
        EconomicBasis  = iv.EconomicBasis;
        Value  = iv.Value;
    }
}

# Args

In [0]:
public record Args(string ReportingNode, int Year, int Month, Periodicity Periodicity, string Scenario){}

In [0]:
public record ImportArgs : Args
{
    public string ImportFormat { get; init; }
       
    public ImportArgs(string reportingNode, int year, int month, Periodicity periodicity, string scenario, string importFormat)
        : base(reportingNode, year, month, periodicity, scenario)
    {
        ImportFormat = importFormat;
    }
}

In [0]:
public record ReportArgs : Args
{
    public string HierarchyName { get; init; }
    
    public CurrencyType CurrencyType { get; init; }
    
    public string ReportName { get; init; } // this is the key to which data to load (like loading behavior). If null, loads everything
    
    public ReportArgs(string reportingNode, int year, int month, Periodicity periodicity, string scenario, string hierarchyName, CurrencyType currencyType)
        : base(reportingNode, year, month, periodicity, scenario)
    {
        CurrencyType = currencyType;
        HierarchyName = hierarchyName;
    }
}